In [34]:
import re
import multiprocessing
import os
import datetime
from collections import defaultdict

In [43]:
line_re = re.compile('^.+ (maildir.+)$')
#date_re = re.compile('Date: (Mon|Tue|Wed|Thu|Fri|Sat|Sun), (?P<date>\d{1,2} [A-Z][a-z]{1,2} \d{4} \d\d:\d\d:\d\d (\-|\+)\d\d\d\d)')
date_re = re.compile('Date: (Mon|Tue|Wed|Thu|Fri|Sat|Sun), (?P<date>\d{1,2} [A-Z][a-z]{1,2} \d{4})')
enron_fn = '/mnt/storage/hex/projects/clnccr/polcon/enron'

In [ ]:
s = "-rw-r--r-- wcohen/users   14236 2004-02-04 01:35 maildir/heard-m/deleted_items/48."

In [4]:
#-rw-r--r-- wcohen/users   14236 2004-02-04 01:35 maildir/heard-m/deleted_items/48.
#fns = []
fn_date = []
with open(enron_fn) as r:  #  + '_small'
    for line in r:
        if line.strip().endswith('.'):
            #print line
            email = os.path.join('/mnt/storage/hex/projects/clnccr/polcon/',
                                 line_re.match(line).group(1))
            with open(email) as r:
                count = 0
                next(r, None)
                date = date_re.search(r.next()).groupdict()['date']
                fn_date.append((email, date))

In [32]:
with open('/mnt/storage/hex/projects/clnccr/polcon/maildir/campbell-l/north_crawar_remediation/3.') as r:
    next(r, None)
    date = date_re.search(r.next())
    print date.groupdict()['date']

31 Aug 2000


In [24]:
fn_date_dict = defaultdict(list)
for fn, date in fn_date:
    fn_split = fn.split('/')[8:]
    custodian = fn_split[0]
    folder = '/'.join(fn_split[1:-1])
    filename = fn_split[-1]
    key = custodian, folder, date
    fn_date_dict[key].append(filename)
fn_date_dict = dict(fn_date_dict)

In [25]:
date_re.search(d).groupdict()['date']

'31 Aug 2000'

In [ ]:
def documents_out(custodian, dir_, filenames):
    out = []
    for filename in filenames:
        with open(os.path.join(maildir, custodian, dir_, filename)) as r:
            txt = f.read()
            out.append((filename, txt))
    return out

In [35]:
body = 'Message-ID: <18322716.1075858011439.JavaMail.evans@thyme>\r\nDate: Thu, 17 Feb 2000 09:59:00 -0800 (PST)\r\nFrom: greg.whalley@enron.com\r\nTo: john.sherriff@enron.com\r\nSubject: Re: Announcement of EnronCredit.com\r\nCc: bryan.seyfried@enron.com\r\nMime-Version: 1.0\r\nContent-Type: text/plain; charset=us-ascii\r\nContent-Transfer-Encoding: 7bit\r\nBcc: bryan.seyfried@enron.com\r\nX-From: Greg Whalley\r\nX-To: John Sherriff\r\nX-cc: Bryan Seyfried\r\nX-bcc: \r\nX-Folder: \\Lawrence_Whalley_Dec2000\\Notes Folders\\Sent\r\nX-Origin: Whalley-L\r\nX-FileName: gwhalley.nsf\r\n\r\nwhat does this new website look like?  can i see a demo version?\n\nwhat have we defined as current limits?'

In [38]:
id_re = re.compile('^Message-ID: (?P<id>.+)\r\n')
from_re = re.compile('\r\nFrom: (?P<from>.+)\r\n')
to_re = re.compile('\r\nTo: (?P<to>.+)\r\n')
subj_re = re.compile('\r\nSubject: (?P<subject>.+)\r\n')
cc_re = re.compile('\r\nCc: (?P<cc>.+)\r\n')
bcc_re = re.compile('\r\nBcc: (?P<bcc>.+)\r\n')
body_re = re.compile('X-FileName: .+\r\n\r\n')

In [40]:
padding_re = re.compile('^\d ')
date_format = '%d %b %Y %X'

In [ ]:
import sqlite3
#import calendar
conn = sqlite3.connect('/mnt/storage/hex/projects/clnccr/polcon/www/enron.db')
c = conn.cursor()
#c.execute('''DROP TABLE emails;''')
# Create table
c.execute('''CREATE TABLE emails
             (id INTEGER PRIMARY KEY AUTOINCREMENT,
              custodian text,
              dir text,
              date text,
              filename text,
              message_id text,
              from_ text,
              to_ text,
              subject text,
              cc text,
              bcc text,
              body text)''')

maildir = '/mnt/storage/hex/projects/clnccr/polcon/maildir/'
# Insert a row of data
# Larger example that inserts many records at a time
data = []
for k, filenames in enron_dict.items():
    custodian, dir_, date = k
    for filename in filenames:
        with open(os.path.join(maildir, custodian, dir_, filename)) as r:
            body = r.read().decode('latin1')
            id_ = id_re.search(body).groupdict()['id']
            from_m = from_re.search(body)
            from_ = from_m.groupdict()['from'] if from_m else ''
            to_m = to_re.search(body)
            to = to_m.groupdict()['to'] if to_m else ''
            subj_m = subj_re.search(body)
            subj = subj_m.groupdict()['subject'] if subj_m else ''
            cc_m = cc_re.search(body)
            cc = cc_m.groupdict()['cc'] if cc_m else ''
            bcc_m = bcc_re.search(body)
            bcc = bcc_m.groupdict()['bcc'] if bcc_m else ''
            body = body_re.split(body)[-1]
            # padding
#             if padding_re.match(date):
#                 date = '0' + date
#             date_ = datetime.datetime.strptime(date[:-6], date_format)
#             # fixing time zones, now all UTC
#             hours = datetime.timedelta(hours=int(date[-3]))
#             if date[-5] == '-':
#                 date_ -= hours
#             else:
#                 date_ += hours
#             # turn to timestap
#             date_ = calendar.timegm(date_.timetuple())
            data.append((custodian, dir_, date, filename, id_, from_, to, subj, cc, bcc, body))

c = conn.cursor()
c.executemany('INSERT INTO emails VALUES (?,?,?,?,?,?,?,?,?,?,?,?)', ([i] + list(d) for i, d in enumerate(data)))

conn.commit()
conn.close()

c.execute('ALTER TABLE emails ADD COLUMN tfidf text;')
# had to make another table ...
c.execute('CREATE TABLE tfidfs (id integer, words text)')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

X = [d[-1] + d[-4].encode('utf8') for d in data]
tf_idf = TfidfVectorizer(input='content', stop_words='english')
X_vect = tf_idf.fit(X)
X_vect_ = X_vect.transform(X)

feats = X_vect.get_feature_names()
num = 5
top_tfidfs = []
for r in range(X_vect_.shape[0]):
    if r % 2500 == 0:
        # add top-num tfidf scoring tokens
        #c.executemany('UPDATE emails SET tfidf=? WHERE id=?', top_tfidfs)
        c.executemany('INSERT INTO tfidfs (words, id) VALUES (?, ?)', top_tfidfs)
        top_tfidfs = []
    tmp = X_vect_[r].toarray().ravel()
    num_ = tmp.nonzero()[0].size
    num_ = num if num_ > num else num_
    top = np.argpartition(-tmp.ravel(), num_)[:num_]
    top_tfidfs.append((' '.join([feats[t] for t in top]), r))

In [ ]:
import glob
X = []
y = []
enron_span = '/home/user/makarov/hexmnt/polcon/enron_spam'
for i in range(1,7,1):
    dir_ = os.path.join(enron_span, 'enron' + str(i))
    for label, num_label in [('spam', 1), ('ham', 0)]:
        tmp = [open(fn).read().decode('latin1') for fn in glob.glob(os.path.join(dir_, label) + '/*txt')]
        X += tmp
        y += [num_label] * len(tmp)

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score
from sklearn.pipeline import make_pipeline
from sklearn.externals import joblib

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
vect = CountVectorizer(input='content', stop_words='english')

X_train_vect = vect.fit_transform(X_train)
sgd = SGDClassifier(loss='log', verbose=True)
sgd.fit(X_train_vect, y_train)

X_test_vect = vect.transform(X_test)
pred = sgd.predict(X_test_vect)
print f1_score(pred, y_test)
# out-of-sample f1-score 0.984665290475

# retrain on all data
pipeline = make_pipeline(CountVectorizer(input='content', stop_words='english'),
                         SGDClassifier(loss='log', verbose=True))
pipeline.fit_transform(X, y)

joblib.dump(pipeline, os.path.join(enron_span, 'sgd.pkl'))
pipeline = joblib.load(os.path.join(enron_span, 'sgd.pkl'))

In [46]:
c.execute('ALTER TABLE tfidfs ADD COLUMN spam boolean;')
c.executemany('UPDATE tfidfs SET spam=? WHERE id=?', enumerate(predictions))

u'/mnt/storage/hex/users/makarov'